In [33]:
import numpy as np

def demographic_parity_difference(predictions, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)

    # print(unique_labels)
    # print(sensitive_attribute)
    
    # store dpd
    dp_diff = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)

    
    # compute dpd for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute proportions of positive outcomes for each group
        group_proportions = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute proportion of positive outcomes for this group
            positive_proportion = np.mean(predictions[group_indices] == label)
            group_proportions.append(positive_proportion)
        
        # compute dpd for this label
        dp_diff[i] = max(group_proportions) - min(group_proportions)

        # ID privileged gender for this label
        privileged_index = np.argmax(group_proportions)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return dp_diff, privileged_gender


In [34]:
import numpy as np

def equal_opportunity_difference(predictions, true_labels, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)
    
    # store eod
    eo_diff = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)
    
    # compute eod for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute TPR for each group
        tpr_group = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute TPR for this group
            true_positives = np.sum((predictions[group_indices] == label) & (true_labels[group_indices] == label))
            actual_positives = np.sum(true_labels[group_indices] == label)
            
            if actual_positives > 0:
                true_positive_rate = true_positives / actual_positives
                tpr_group.append(true_positive_rate)
            else:
                tpr_group.append(0.0)  # division by zero case
        
        # compute eod for this label
        eo_diff[i] = max(tpr_group) - min(tpr_group)
        
        # ID privileged gender for this label
        privileged_index = np.argmax(tpr_group)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return eo_diff, privileged_gender


In [35]:
import numpy as np

def average_odds_difference(predictions, true_labels, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)
    
    # store aod
    aod_diff = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)
    
    # compute aod for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute FPR and TPR for each group
        fpr_group = []
        tpr_group = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute FPR for this group
            false_positives = np.sum((predictions[group_indices] == label) & (true_labels[group_indices] != label))
            actual_negatives = np.sum(true_labels[group_indices] != label)
            if actual_negatives > 0:
                false_positive_rate = false_positives / actual_negatives
                fpr_group.append(false_positive_rate)
            else:
                fpr_group.append(0.0)  # division by zero case
            
            # compute TPR for this group
            true_positives = np.sum((predictions[group_indices] == label) & (true_labels[group_indices] == label))
            actual_positives = np.sum(true_labels[group_indices] == label)
            if actual_positives > 0:
                true_positive_rate = true_positives / actual_positives
                tpr_group.append(true_positive_rate)
            else:
                tpr_group.append(0.0)  # division by zero case
        
        # compute aod for this label
        aod_diff[i] = (max(fpr_group) - min(fpr_group) + max(tpr_group) - min(tpr_group)) / 2
        
        # ID privileged gender for this label
        privileged_index = np.argmax(tpr_group)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return aod_diff, privileged_gender


In [36]:
import numpy as np

def disparate_impact(predictions, sensitive_attribute):

    # convert preds and sensitive attribute to arrays
    predictions = np.array(predictions)
    sensitive_attribute = np.array(sensitive_attribute)
    
    # get labels + sensitive attribute vals
    unique_labels = np.unique(predictions)
    unique_sensitive_attribute = np.unique(sensitive_attribute)
    
    # store di ratio
    di_ratio = np.zeros(len(unique_labels))
    privileged_gender = np.empty(len(unique_labels), dtype=object)
    
    # compute di ratio for each label
    for i, label in enumerate(unique_labels):
        # find indices where pred is equal to current label
        label_indices = np.where(predictions == label)[0]
        
        # compute proportions of positive outcomes for each group
        favorable_proportions = []
        for a in unique_sensitive_attribute:
            # find indices where sensitive attribute is equal to current value
            group_indices = np.where(sensitive_attribute == a)[0]
            
            # compute proportion of positive outcomes for this group
            favorable_proportion = np.mean(predictions[group_indices] == label)
            favorable_proportions.append(favorable_proportion)
        
        # compute DI ratio for this label
        di_ratio[i] = favorable_proportions[1] / favorable_proportions[0] if favorable_proportions[0] != 0 else 0
        
        # ID privileged gender for this label
        privileged_index = np.argmax(favorable_proportions)
        privileged_gender[i] = unique_sensitive_attribute[privileged_index]
    
    return di_ratio, privileged_gender


Data


In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


data = pd.read_csv('thesis_dataset.csv')

# drop rows with missing values in text columns
text_columns = ['Name', 'Education', 'Work Experience', 'Skills', 'Awards']
data = data.dropna(subset=text_columns)

# extract text data and labels
X_text = data['Name'] + ' ' + data['Education'] + ' ' + data['Work Experience'] + ' ' + data['Skills'] + ' ' + data['Awards']
y = data['Fit']
gender = data['Gender']

# vectorize the text data with TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_text_vec = vectorizer.fit_transform(X_text)

# split data
X_train, X_test, y_train, y_test, gender_train, gender_test = train_test_split(X_text_vec, y, gender, test_size=0.2, random_state=42)



SVM


In [38]:
# train SVM

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42) 
svm_classifier.fit(X_train, y_train)

# predict on test set
y_pred = svm_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))

Demographic parity difference for each label: (array([0.06445221, 0.03275058, 0.12587413, 0.03286713, 0.0041958 ]), array(['male', 'male', 'female', 'male', 'female'], dtype=object))
Equal opportunity difference for each label: (array([0.27972028, 0.13392857, 0.12121212, 0.08235294, 0.07333333]), array(['male', 'female', 'female', 'male', 'female'], dtype=object))
Average odds difference for each label: (array([0.15741779, 0.07440174, 0.0934891 , 0.06322135, 0.04872274]), array(['male', 'female', 'female', 'male', 'female'], dtype=object))
Disparate impact for each label: (array([1.34030769, 1.25429864, 0.6043956 , 1.24102564, 0.98153846]), array(['male', 'male', 'female', 'male', 'female'], dtype=object))

Accuracy: 0.71
              precision    recall  f1-score   support

     average       0.55      0.67      0.60        48
         bad       0.84      0.73      0.78        44
        good       0.72      0.62      0.66        78
    very bad       0.78      0.84      0.81        

Log Reg

In [30]:

# train the Logistic Regression classifier

logistic_classifier = LogisticRegression(max_iter=1000, random_state=42)
logistic_classifier.fit(X_train, y_train)

# predict on test set
y_pred = logistic_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)


# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))


['average' 'bad' 'good' 'very bad' 'very good']
Demographic parity difference for each label: (array([0.09452214, 0.04801865, 0.11095571, 0.002331  , 0.03391608]), array(['male', 'male', 'female', 'male', 'female'], dtype=object))
Equal opportunity difference for each label: (array([0.34265734, 0.10714286, 0.08484848, 0.03529412, 0.08666667]), array(['male', 'female', 'female', 'female', 'female'], dtype=object))
Average odds difference for each label: (array([0.19832029, 0.05593789, 0.0747148 , 0.03076337, 0.07309969]), array(['male', 'female', 'female', 'female', 'female'], dtype=object))
Disparate impact for each label: (array([1.65668016, 1.39615385, 0.6145749 , 1.01538462, 0.8852071 ]), array(['male', 'male', 'female', 'male', 'female'], dtype=object))
Accuracy: 0.68
              precision    recall  f1-score   support

     average       0.52      0.54      0.53        48
         bad       0.79      0.68      0.73        44
        good       0.72      0.56      0.63        78


Random Forest

In [31]:
# train Random Forest

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# predict on test set
y_pred = rf_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))

['average' 'bad' 'good' 'very bad' 'very good']
Demographic parity difference for each label: (array([0.01759907, 0.01759907, 0.07983683, 0.04067599, 0.0039627 ]), array(['male', 'male', 'female', 'male', 'male'], dtype=object))
Equal opportunity difference for each label: (array([0.20979021, 0.19642857, 0.07070707, 0.09117647, 0.00666667]), array(['male', 'female', 'male', 'male', 'male'], dtype=object))
Average odds difference for each label: (array([0.10506981, 0.10996209, 0.05952702, 0.07205789, 0.02506231]), array(['male', 'female', 'male', 'male', 'male'], dtype=object))
Disparate impact for each label: (array([1.12226721, 1.12226721, 0.74296435, 1.28259109, 1.01538462]), array(['male', 'male', 'female', 'male', 'male'], dtype=object))
Accuracy: 0.83
              precision    recall  f1-score   support

     average       0.90      0.75      0.82        48
         bad       0.82      0.75      0.79        44
        good       0.89      0.81      0.85        78
    very bad    

Grad Boost

In [32]:
# train Gradient Boosting classifier

gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)  
gb_classifier.fit(X_train, y_train)

# predict on test set
y_pred = gb_classifier.predict(X_test)

dp_diff = demographic_parity_difference(y_pred, gender_test)
print("Demographic parity difference for each label:", dp_diff)

eo_diff = equal_opportunity_difference(y_pred, y_test, gender_test)
print("Equal opportunity difference for each label:", eo_diff)

ao_diff = average_odds_difference(y_pred, y_test, gender_test)
print("Average odds difference for each label:", ao_diff)

di_ratio = disparate_impact(y_pred, gender_test)
print("Disparate impact for each label:", di_ratio)

# evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# classification report
print(classification_report(y_test, y_pred))


['average' 'bad' 'good' 'very bad' 'very good']
Demographic parity difference for each label: (array([0.07960373, 0.05582751, 0.0955711 , 0.02062937, 0.01923077]), array(['male', 'male', 'female', 'female', 'female'], dtype=object))
Equal opportunity difference for each label: (array([0.22377622, 0.08928571, 0.02828283, 0.06764706, 0.05333333]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Average odds difference for each label: (array([0.14857574, 0.04776997, 0.03197528, 0.03816929, 0.06806854]), array(['male', 'female', 'female', 'female', 'male'], dtype=object))
Disparate impact for each label: (array([1.45685619, 1.43348416, 0.66801619, 0.87032967, 0.92307692]), array(['male', 'male', 'female', 'female', 'female'], dtype=object))
Accuracy: 0.83
              precision    recall  f1-score   support

     average       0.71      0.83      0.77        48
         bad       0.88      0.82      0.85        44
        good       0.92      0.74      0.82        78
